In [16]:

import numpy as np
import pandas as pd   
import os
from pathlib import Path
import glob
import json
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import nltk
import cv2
import matplotlib.pyplot as plt
import random
from tensorflow.keras.layers.experimental import preprocessing
import pickle

print('fim')
percentageData = "3"
file_to_search = '/home/jupyter/imported/'

fim


In [17]:
from os.path import isfile, join

def GravarArquivo ( data_dict,fname):
    if len(data_dict) <=0:
        print("branco")
    
    elif os.path.isfile(fname):
    # File exists
     with open(fname, 'a+') as outfile:
        outfile.seek(outfile.tell() - 2, os.SEEK_SET)
        outfile.truncate()
        outfile.write(',')
        outfile.write(json.dumps(data_dict,ensure_ascii=False, indent=4)[1:-1])
        outfile.write(']')
        outfile.close() 
    else: 
    # Create file
     with open(fname, 'w') as outfile:
        json.dump(data_dict, outfile, ensure_ascii=False, indent=4) 
        outfile.close()


In [18]:
def GravarArquivo2 ( data_dict,fname):
    if len(data_dict) <=0:
        print("branco")
        
    elif os.path.isfile(fname):
    # File exists
     with open(fname, 'a+') as outfile:
        outfile.seek(outfile.tell() - 2, os.SEEK_SET)
        outfile.truncate()
        outfile.write(',')
        outfile.write(json.dumps(data_dict,ensure_ascii=False, indent=4)[1:-1])
        outfile.write(']')
        outfile.close() 
    else: 
    # Create file
     with open(fname, 'w') as outfile:
        json.dump(data_dict, outfile, ensure_ascii=False, indent=4) 
        

In [19]:
with open('Percentage_'+ str(percentageData) +  '/train_dataframe', "rb") as f:
     train_dataframe =  pickle.load(f) 
with open('Percentage_'+ str(percentageData) +  '/encoder', "rb") as f:
     encoder =  pickle.load(f)
with open('Percentage_'+ str(percentageData) +  '/val_dataframe', "rb") as f:
     val_dataframe =  pickle.load(f)

In [20]:
def validaRede (model ,memory ) : 
                index=1
                #0 - path image 
                #1 - address 
                ListasAcertasTrainlist = []
                QuestoesCertasList =[]
                QuestoesErradasList=[]
                QTDERRADA =0 
                QTDCERTA=0
                for contador in range(len(train_dataframe)) :

                    im=cv2.imread(train_dataframe.iloc[contador]['Path'])
                    im=cv2.resize(im,(200,200))
                    q=train_dataframe.iloc[contador]['Question']  
                    q=encoder.encode(q)
                    paddings = [[0, 50-tf.shape(q)[0]]]
                    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
                    q=np.array(q)
                    im.resize(1,200,200,3)
                    q.resize(1,50)
                    ans=model.predict([im,q]) 
                    decodAns = encoder.decode([np.argmax(ans)])
                    if train_dataframe.iloc[contador]['Answer'] != decodAns :
                        ListasAcertasTrainlist.append(0)
                        QuestoesErradasList.append("questão nr: " + str(contador))
                        QuestoesErradasList.append(train_dataframe.iloc[contador]['Question'])
                        QuestoesErradasList.append( "certa:" + train_dataframe.iloc[contador]['Answer'] + " errada : " + decodAns) 
                        QTDERRADA = QTDERRADA+1
                        
                    else:     
                        ListasAcertasTrainlist.append(1)
                        QuestoesCertasList.append("questão nr: " + str(contador))
                        QuestoesCertasList.append(train_dataframe.iloc[contador]['Question'])
                        QuestoesCertasList.append( "certa:" + train_dataframe.iloc[contador]['Answer'] + " preditada : " + decodAns) 
                        QTDCERTA = QTDCERTA+1
                       
                    if (contador%2000) == 0 : 
                        GravarArquivo (ListasAcertasTrainlist,FolderSource  + '/ResultPredication_Train_' +memory +'.json') 
                        GravarArquivo2 (QuestoesErradasList,FolderSource  + '/ResultPredication_Train_' +memory +'_Ques_errada.json')  
                        GravarArquivo2 (QuestoesCertasList,FolderSource  + '/ResultPredication_Train_' +memory +'_Ques_certa.json')  
                        ListasAcertasTrainlist=[]
                        QuestoesCertasList =[]
                        QuestoesErradasList=[]
                QuestoesErradasList.append("QTD de questões : " + str(QTDERRADA))
                QuestoesCertasList.append("QTD de questões :  " + str(QTDCERTA))
                
                if (contador%2000) >= 0 :
                    GravarArquivo (ListasAcertasTrainlist,FolderSource +  '/ResultPredication_Train_' +memory +'.json')   
                    GravarArquivo2 (QuestoesErradasList,FolderSource +  '/ResultPredication_Train_' +memory +'_Ques_errada.json')    
                    GravarArquivo2 (QuestoesCertasList,FolderSource +  '/ResultPredication_Train_' +memory +'_Ques_certa.json')  
                   
                ListasValidacaolist = []
                QuestoesCertasList =[]
                QuestoesErradasList=[]
                QTDERRADA = 0
                QTDCERTA = 0
                for contador in   range(len(val_dataframe)) :

                    im=cv2.imread(val_dataframe.iloc[contador]['Path'])
                    im=cv2.resize(im,(200,200))
                    q=val_dataframe.iloc[contador]['Question'] 
                    q=encoder.encode(q)
                    paddings = [[0, 50-tf.shape(q)[0]]]
                    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
                    q=np.array(q)
                    im.resize(1,200,200,3)
                    q.resize(1,50)

                    ans=model.predict([im,q]) 
                    decodAns = encoder.decode([np.argmax(ans)])
                    if val_dataframe.iloc[contador]['Answer'] != decodAns :
                        ListasValidacaolist.append(0)
                        QuestoesErradasList.append("questão nr: " + str(contador))
                        QuestoesErradasList.append(val_dataframe.iloc[contador]['Question'])
                        QuestoesErradasList.append( "certa:" + val_dataframe.iloc[contador]['Answer'] + " errada : " + decodAns) 
                        QTDERRADA = QTDERRADA+1
                    else:     
                        ListasValidacaolist.append(1)
                        QuestoesCertasList.append("questão nr: " + str(contador))
                        QuestoesCertasList.append(val_dataframe.iloc[contador]['Question'])
                        QuestoesCertasList.append( "certa:" + val_dataframe.iloc[contador]['Answer'] + " preditada : " + decodAns) 
                        QTDCERTA = QTDCERTA+1
                        
                    if (contador%2000) == 0 : 
                        GravarArquivo (ListasValidacaolist,FolderSource + '/ResultPredication_Val_' +memory +'.json')   
                        GravarArquivo2 (QuestoesCertasList,FolderSource +  '/ResultPredication_Val_' +memory +'_Ques_Certas.json')  
                        GravarArquivo2 (QuestoesErradasList,FolderSource +  '/ResultPredication_Val_' +memory +'_Ques_Errada.json')  
                        QuestoesCertasList =[]
                        QuestoesErradasList=[]               
                        ListasValidacaolist=[]
                        
                QuestoesErradasList.append("QTD de questões : " + str(QTDERRADA))
                QuestoesCertasList.append("QTD de questões :  " + str(QTDCERTA))
                if (contador%2000) >= 0 :      
                      GravarArquivo (ListasValidacaolist,FolderSource + '/ResultPredication_Val_' +memory +'.json')   
                      GravarArquivo2 (QuestoesCertasList,FolderSource +  '/ResultPredication_Val_' +memory +'_Ques_Certas.json')  
                      GravarArquivo2 (QuestoesErradasList,FolderSource +  '/ResultPredication_Val_' +memory +'_Ques_Errada.json')  

In [21]:
model2 = tf.keras.models.load_model('Percentage_'+ str(percentageData) +"/ModelTreinamento")
model2.summary()
file_to_search = '/home/jupyter/imported/'

       

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 256)      26112       text_input[0][0]                 
__________________________________________________________________________________________________
image_input (InputLayer)        [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 50, 512)      1050624     embedding[0][0]                  
______________________________________________________________________________________________

In [22]:
FolderSource = file_to_search  + "Percentage_3" 
validaRede(model2,"fullmemeory")

branco
branco


In [ ]:
print('oi')

folders = dirs = [d for d in os.listdir(file_to_search) if os.path.isdir(os.path.join(file_to_search, d))]



for folder in folders:
    if 'Percentage_0.1'  in folder:
        qtd = 0 
     
        vocab_set=set()#set object used to store the vocabulary
        partSplit = folder.split('_')
        perc = partSplit[1]
        FolderSource = file_to_search + '//' + folder 
        Files = [f for f in os.listdir(FolderSource) if isfile(join(FolderSource, f))]
        posQues =[]
        valList=[]
        trainList=[]
        memorys = []
                     
        for File in Files:
                print (File)
                if 'ckpt' in File and 'index' in File:
                    obj=[]
                    obj.append(perc)
                    obj.append(File[:-6])
                    memorys.append(obj)
            
        
        
        print('memorys ')
        
        for memory in memorys :
            print(memory)
            #'weights-100' in  memory[1]
            if 1==1 :
                print(memory[1])
           

                filepath=FolderSource + '//' + memory[1]  
                print(filepath)
                model2 = tf.keras.models.load_model('Percentage_'+ str(percentageData) +"/ModelTreinamento")   
                
                model2.load_weights(filepath)
                model2.summary()
                validaRede(model2,memory[1])
                
                   
    
                
print('fim')